In [ ]:
import spacy
import re
# import nltk
# from nltk import bigrams
# from nltk import ngrams

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
dist_dict = {}

list_all_tokens = []

list_unique_tokens = []

list_pos_tags = []

In [ ]:
uni_cohesive_markers = ["because", "besides", "but", "consequently", "despite", "except", "further", "furthermore", "hence", "however", "instead", "maybe", "moreover", "nevertheless", "otherwise",  "since", "so", "therefore",  "though", "thus",  "yet", "concerning"]
bi_cohesive_markers = [["as","for"], ["as","to"], ["even", "if"], ["even","though"], ["in", "addition"], ["in","conclusion"], ["in","spite"], ["referring","to"], ["the", "former"], ["the", "latter"], ["this", "implies"]]
tri_cohesive_markers = [["in","other","words"], ["is","to","say"], ["on","account","of"], ["on","the","contrary"], ["with","reference","to"], ["with","regard","to"]]
quat_cohesive_markers = ["on","the","other","hand"]

Universal Dependency Average Summed Distance

In [ ]:
def dist_from_root(tok, count):
  #print("tok ", tok)
  if tok.dep_ == 'ROOT':
    #print("count ", count)
    return count
  else:
    return dist_from_root(tok.head, count+1)

In [ ]:
def summed_dist_for_sent_length(sum_dist, sent_length):
  if sent_length not in dist_dict:
    dist_dict[sent_length] = []
  dist_dict[sent_length].append(sum_dist)

In [ ]:
def summed_dist(sent):
  doc = nlp(sent)
  total_count = 0

  for token in doc:
    total_count += dist_from_root(token,0)
    #print(total_count)

  sent_len = len(sent.split())

  summed_dist_for_sent_length(total_count,sent_len)

In [ ]:
reader = open('para_t5.txt', 'r')

In [ ]:
def collect_all_unique_tokens(sent):
  doc = nlp(sent)
  for token in doc:
    list_all_tokens.append(token)
    list_pos_tags.append(token.pos_)
    if token.lemma_ not in list_unique_tokens:
      list_unique_tokens.append(token.lemma_)

In [ ]:
# count = 0
while True:
  line = reader.readline()
  if not line:
    break
  #line = line.translate(str.maketrans('', '', string.punctuation))
  line = re.sub(r"\s([?.!',](?:\s|$))", r'\1', line)
  collect_all_unique_tokens(line)
  #print(line)
  try:
    summed_dist(line)
  except:
    print("An exception occurred")
  # count += 1
  # if count%5000==0:
  #   print("Now at line: ", str(count))
  # file.write(str(dist))
  # file.write('\n')

Cohesive Markers (Volensky features)

In [ ]:
#print(list_all_tokens)
count_cohesive = 0
#print(uni_cohesive_markers)
for word in list_all_tokens:
  if str(word).lower() in uni_cohesive_markers:
    count_cohesive += 1
print(count_cohesive)

first_word = str(list_all_tokens[0]).lower()
for word in list_all_tokens[1:]:
  bigram = []
  bigram.append(first_word)
  second_word = str(word).lower()
  bigram.append(second_word)
  #print(bigram)
  if bigram in bi_cohesive_markers:
    count_cohesive += 1
  first_word = second_word
print(count_cohesive)

first_word = str(list_all_tokens[0]).lower()
second_word = str(list_all_tokens[1]).lower()
for word in list_all_tokens[2:]:
  trigram = []
  trigram.append(first_word)
  trigram.append(second_word)
  third_word = str(word).lower()
  trigram.append(third_word)
  #print(trigram)
  if trigram in tri_cohesive_markers:
    count_cohesive += 1
  first_word = second_word
  second_word = third_word
print(count_cohesive)

first_word = str(list_all_tokens[0]).lower()
second_word = str(list_all_tokens[1]).lower()
third_word = str(list_all_tokens[2]).lower()
for word in list_all_tokens[3:]:
  quatgram = []
  quatgram.append(first_word)
  quatgram.append(second_word)
  quatgram.append(third_word)
  fourth_word = str(word).lower()
  quatgram.append(fourth_word)
  #print(quatgram)
  if quatgram == quat_cohesive_markers:
    count_cohesive += 1
  first_word = second_word
  second_word = third_word
  third_word = fourth_word
print(count_cohesive)

print("COHESIVE MARKERS: " , count_cohesive)

395
423
439
446
COHESIVE MARKERS:  446


Lexical Richness (TTR) (Rabinovich et al. 2016 on the ENTTT corpus)

In [ ]:
ttr = len(list_unique_tokens) / len(list_all_tokens) # lexical richness / type-to-token ratio (TTR): dividing the number of unique (lemmatized) tokens by the total number of tokens
print(ttr)

0.07355210132412204


In [ ]:
# count_propn = list_pos_tags.count('PROPN')
# print(count_propn)
count_coord = list_pos_tags.count('CCONJ') + list_pos_tags.count('SCONJ')
print(count_coord)

2101


In [ ]:
pos_counts = {}

for i in list_pos_tags:
  pos_counts[i] = pos_counts.get(i, 0) + 1

pos_sort = sorted(pos_counts.items(), key=lambda x: x[0])

with open('pos_counts.txt', 'w') as pf:
  for pos,num in pos_sort:
    out = str(pos) + "," + str(num)
    pf.write(out)
    pf.write("\n")

pf.close()

In [ ]:
def average(lst):
  return sum(lst) / len(lst)

In [ ]:
res = sorted(dist_dict.items(), key=lambda x: x[0])

print(dist_dict)
print(res)

ud_avg_L = [average(x[1]) for x in res]
#print(ud_avg_L)
sent_length_L = [x[0] for x in res]
#print(sent_length_L)

{15: [32, 64, 39, 39, 36, 48, 35, 26, 31, 41, 39, 32, 34, 43, 54, 40, 45, 36, 35, 61, 42, 42, 39, 40, 25, 46, 36, 35, 37, 33, 59, 30, 60, 52, 48, 46, 36, 44, 36, 43, 37, 46, 36, 36, 43, 33, 39, 49, 42, 43, 36, 31, 34, 46, 45, 48, 43, 43, 52, 36, 54, 42, 34, 52, 34, 41, 31, 54, 42, 45, 38, 51, 39, 47, 51, 36, 52, 44, 36, 74, 37, 65, 51, 55, 26, 43, 39, 42, 49, 29, 54, 43, 38, 41, 40, 48, 61, 45, 46, 40, 46, 69, 42, 42, 29, 43, 47, 53, 28, 40, 48, 50, 32, 31, 45, 42, 44, 36, 46, 73, 47, 49, 59, 76, 52, 38, 36, 64, 48, 62, 40, 57, 53, 42, 40, 46, 59, 50, 46, 32, 36, 71, 55, 47, 47, 53, 39, 45, 36, 39, 53, 43, 36, 42, 57, 46, 41, 41, 59, 33, 56, 77, 39, 36, 37, 46, 44, 33, 37, 62], 18: [56, 43, 66, 50, 64, 60, 70, 63, 46, 70, 68, 57, 52, 56, 59, 58, 36, 77, 55, 61, 69, 77, 36, 57, 57, 49, 52, 42, 42, 41, 65, 74, 63, 54, 48, 54, 62, 48, 66, 62, 52, 58, 53, 51, 51, 51, 62, 70, 72, 65, 71, 45, 62, 50, 63, 53, 48, 31, 57, 70, 49, 71, 55, 34, 62, 48, 43, 59, 63, 46, 77, 59, 82, 34, 52, 56, 76, 

In [ ]:
with open('sent_length.txt', 'w') as fp:
    for item in sent_length_L:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

fp.close()

Done


In [ ]:
with open('ud_avg.txt', 'w') as ud:
    for item in ud_avg_L:
        # write each item on a new line
        ud.write("%s\n" % item)
    print('Done')

ud.close()

Done


In [ ]:
reader.close()